## 市のオープンデータの緯度経度取得スクリプト作成 #7

1. 対象ウェブサイトが施設カテゴリ毎に下記の構造を持っていたため、loopを回してテーブル情報を取得
    - 保育園: https://www.city.funabashi.lg.jp/funakkonavi/map/0/index.html
    - 幼稚園: https://www.city.funabashi.lg.jp/funakkonavi/map/1/index.html
    - ...
    - その他預け先: https://www.city.funabashi.lg.jp/funakkonavi/map/12/index.html
    
2. それぞれのテーブルの住所に対してgoogle geocode APIを利用して経度・緯度を取得

In [31]:
from bs4 import BeautifulSoup

url=f"https://www.city.funabashi.lg.jp/funakkonavi/map/0/index.html"
context = ssl._create_unverified_context()
response = request.urlopen(url, context=context)
html = response.read()

soup = BeautifulSoup(html, 'html.parser')

# 施設カテゴリ
res = soup.select("select#category-map")

# [<option selected="selected" value="">選択してください</option>,を除く
results = res[0].find_all()

id2CategoryType = {idx:t.text for idx, t in enumerate(results[1:])}

In [92]:
id2CategoryType

{0: '保育園',
 1: '幼稚園',
 2: '認定こども園',
 3: '小規模保育事業所',
 4: '家庭的保育者',
 5: '事業所内保育事業所',
 6: '認証保育所',
 7: '認可外保育施設',
 8: '一時保育',
 9: '病児・病後児保育',
 10: '児童ホーム',
 11: '子育て支援センター',
 12: 'その他預け先'}

In [33]:
import pandas as pd
from urllib import request
import ssl
# , flavor="bs4"
# private PCジャないからか、SSL errorが発生する。定期実行をする際はよくないけど下記のやり方でスクレーピングする。

all_facilities = {}
for i in range(13):
    url=f"https://www.city.funabashi.lg.jp/funakkonavi/map/{i}/index.html"
    context = ssl._create_unverified_context()
    response = request.urlopen(url, context=context)
    html = response.read()
    category = id2CategoryType[i]
    print(category)
    try:
        facilities = pd.read_html(html, )
    except ValueError as e:
        print(f"ValueError: No tables found for https://www.city.funabashi.lg.jp/funakkonavi/map/{i}/index.html")
        facilities = []
    all_facilities[category] = facilities


保育園
幼稚園
認定こども園
小規模保育事業所
家庭的保育者
事業所内保育事業所
ValueError: No tables found for https://www.city.funabashi.lg.jp/funakkonavi/map/5/index.html
認証保育所
認可外保育施設
一時保育
病児・病後児保育
児童ホーム
子育て支援センター
その他預け先
ValueError: No tables found for https://www.city.funabashi.lg.jp/funakkonavi/map/12/index.html


In [35]:
all_facilities["児童ホーム"]

[          施設名                          住所      電話番号(代表)
 0     前原児童ホーム    〒274-0825千葉県船橋市前原西6-1-12  047-475-5454
 1    高根台児童ホーム     〒274-0065千葉県船橋市高根台2-2-2  047-466-3638
 2   習志野台児童ホーム    〒274-0063千葉県船橋市習志野台3-4-1  047-464-0001
 3    金杉台児童ホーム     〒273-0852千葉県船橋市金杉台2-2-4  047-447-7837
 4     若松児童ホーム      〒273-0013千葉県船橋市若松2-3-6  047-433-7991
 5     西船児童ホーム       〒273-0033千葉県船橋市本郷町554  047-333-2007
 6     小室児童ホーム      〒270-1471千葉県船橋市小室町3308  047-457-1000
 7     三山児童ホーム     〒274-0072千葉県船橋市三山2-42-3  047-478-6800
 8   八木が谷児童ホーム    〒274-0804千葉県船橋市みやぎ台1-7-1  047-448-4400
 9    松が丘児童ホーム   〒274-0064千葉県船橋市松が丘1-52-22  047-463-5087
 10   飯山満児童ホーム  〒274-0822千葉県船橋市飯山満町2-488-8  047-468-0411
 11    夏見児童ホーム    〒273-0865千葉県船橋市夏見4-39-15  047-423-5999
 12    塚田児童ホーム    〒273-0042千葉県船橋市前貝塚町601-1  047-439-9299
 13    宮本児童ホーム     〒273-0003千葉県船橋市宮本6-18-1  047-424-9842
 14    三咲児童ホーム     〒274-0812千葉県船橋市三咲3-5-10  047-448-2397
 15   新高根児童ホーム    〒274-0814千葉県船橋市新高根1-12-9  047-469-3940
 16   薬円台児童ホーム    〒274-0077千葉県船

In [42]:
# !pip install pygeocoder
# !pip install googlemaps

In [43]:
import googlemaps

In [46]:
googleapikey = os.environ["GOOGLE_API_KEY"] 

In [47]:
import pandas as pd
import urllib
import urllib.error
import urllib.request

# Google API モジュール
from pygeocoder import Geocoder
import googlemaps

In [54]:
gmaps = googlemaps.Client(key=googleapikey)

In [55]:
# gmaps.geocode(address)

In [58]:
r = all_facilities["児童ホーム"][0]["住所"].map(lambda loc: gmaps.geocode(loc))

In [80]:
# r[0][0]['address_components']
# # len(r[0][0])
# r[0][0].keys()

In [75]:
# こんな感じでデータ収集完了
r[0][0]['geometry']["viewport"]

{'northeast': {'lat': 35.7040551802915, 'lng': 140.0267875802915},
 'southwest': {'lat': 35.70135721970851, 'lng': 140.0240896197085}}

In [78]:
all_facilities["児童ホーム"][0]["geocodes"] = r.map(lambda x: x[0]['geometry']["viewport"], na_action="ignore")

In [89]:
# all_facilities["児童ホーム"][0]

In [ ]:
from IPython.display import display

In [90]:
for k in all_facilities.keys():
    print(k)
    print(len(all_facilities[k]))
    if len(all_facilities[k]) == 0:
        continue
    r = all_facilities[k][0]["住所"].map(lambda loc: gmaps.geocode(loc))
    all_facilities[k][0]["geocodes"] = r.map(lambda x: x[0]['geometry']["viewport"], na_action="ignore")
    display(all_facilities[k][0])

保育園
1


,施設名,住所,電話番号(代表),geocodes
0,宮本第一保育園,〒273-0003千葉県船橋市宮本6-26-27,047-442-5639,"{'northeast': {'lat': 35.6992823802915, 'lng':..."
1,宮本第二保育園,〒273-0003千葉県船橋市宮本8-9-10,047-423-1304,"{'northeast': {'lat': 35.6928209802915, 'lng':..."
2,若松保育園,〒273-0013千葉県船橋市若松2-6-3,047-433-4041,"{'northeast': {'lat': 35.6808319302915, 'lng':..."
3,湊町保育園,〒273-0011千葉県船橋市湊町1-16-23,047-431-4961,"{'northeast': {'lat': 35.6941728802915, 'lng':..."
4,千鳥保育園,〒273-0004千葉県船橋市南本町13-1,047-431-5061,"{'northeast': {'lat': 35.69943578029149, 'lng'..."
...,...,...,...,...
118,ミアヘルサ保育園ひびき新船橋北,〒273-0045 船橋市山手1-1-11,電話：047-498-9370 FAX：047-498-9371,"{'northeast': {'lat': 35.7161008, 'lng': 139.9..."
119,船橋馬込沢雲母保育園,〒273-0046 船橋市上山町2-487-1,電話：047-430-2424 FAX：047-430-2425,"{'northeast': {'lat': 35.73759818029149, 'lng'..."
120,船橋こどもの木保育園,〒274-0815 船橋市西習志野2-6-1,電話：047-468-8207 FAX：047-468-8208,"{'northeast': {'lat': 35.7268150802915, 'lng':..."
121,京進のほいくえん HOPPA前原西,〒274-0825 船橋市前原西4-12-15,電話：047-489-5260 FAX：047-489-5261,"{'northeast': {'lat': 35.6997823802915, 'lng':..."


幼稚園
1


,施設名,住所,電話番号(代表),geocodes
0,恵楓幼稚園,〒273-0003 千葉県船橋市宮本8-35-12,047-424-5100,"{'northeast': {'lat': 35.6922564802915, 'lng':..."
1,若松幼稚園,〒273-0013 千葉県船橋市若松2-3-3,047-431-7751,"{'northeast': {'lat': 35.6799572802915, 'lng':..."
2,船橋幼稚園,〒273-0005 千葉県船橋市本町3-36-32,047-424-3351,"{'northeast': {'lat': 35.6983850302915, 'lng':..."
3,ひなぎく幼稚園,〒273-0021 千葉県船橋市海神4-29-10,047-431-3569,"{'northeast': {'lat': 35.71262428029149, 'lng'..."
4,二葉幼稚園,〒273-0021 千葉県船橋市海神5-7-14,047-431-4701,"{'northeast': {'lat': 35.7085295802915, 'lng':..."
5,海神幼稚園,〒273-0021 千葉県船橋市海神6-18-34,047-431-5708,"{'northeast': {'lat': 35.7066712802915, 'lng':..."
6,山野幼稚園,〒273-0031 千葉県船橋市西船2-10-7,047-431-2431,"{'northeast': {'lat': 35.71180248029149, 'lng'..."
7,西船幼稚園,〒273-0031 千葉県船橋市西船2-32-25,047-431-2932,"{'northeast': {'lat': 35.7178237802915, 'lng':..."
8,葛飾幼稚園,〒273-0031 千葉県船橋市西船4-16-4,047-431-4463,"{'northeast': {'lat': 35.7118348802915, 'lng':..."
9,船橋ひかり幼稚園,〒273-0034 千葉県船橋市二子町498,047-335-5998,"{'northeast': {'lat': 35.7123739802915, 'lng':..."


認定こども園
1


...

児童ホーム
1


,施設名,住所,電話番号(代表),geocodes
0,前原児童ホーム,〒274-0825千葉県船橋市前原西6-1-12,047-475-5454,"{'northeast': {'lat': 35.7040551802915, 'lng':..."
1,高根台児童ホーム,〒274-0065千葉県船橋市高根台2-2-2,047-466-3638,"{'northeast': {'lat': 35.7356971, 'lng': 140.0..."
2,習志野台児童ホーム,〒274-0063千葉県船橋市習志野台3-4-1,047-464-0001,"{'northeast': {'lat': 35.7232664302915, 'lng':..."
3,金杉台児童ホーム,〒273-0852千葉県船橋市金杉台2-2-4,047-447-7837,"{'northeast': {'lat': 35.73725478029149, 'lng'..."
4,若松児童ホーム,〒273-0013千葉県船橋市若松2-3-6,047-433-7991,"{'northeast': {'lat': 35.6806119802915, 'lng':..."
5,西船児童ホーム,〒273-0033千葉県船橋市本郷町554,047-333-2007,"{'northeast': {'lat': 35.70961018029149, 'lng'..."
6,小室児童ホーム,〒270-1471千葉県船橋市小室町3308,047-457-1000,"{'northeast': {'lat': 35.78942078029149, 'lng'..."
7,三山児童ホーム,〒274-0072千葉県船橋市三山2-42-3,047-478-6800,"{'northeast': {'lat': 35.6977265802915, 'lng':..."
8,八木が谷児童ホーム,〒274-0804千葉県船橋市みやぎ台1-7-1,047-448-4400,"{'northeast': {'lat': 35.75772188029149, 'lng'..."
9,松が丘児童ホーム,〒274-0064千葉県船橋市松が丘1-52-22,047-463-5087,"{'northeast': {'lat': 35.7373523802915, 'lng':..."


子育て支援センター
1


,施設名,住所,電話番号(代表),geocodes
0,南本町子育て支援センター,〒273-0004千葉県船橋市南本町10-1,047-434-3910・【相談専用】047-435-8333,"{'northeast': {'lat': 35.69850168029149, 'lng'..."
1,高根台子育て支援センター,〒274-0065千葉県船橋市高根台2-1-1,047-466-5666・【相談専用】047-466-3633,"{'northeast': {'lat': 35.7362657802915, 'lng':..."


その他預け先
0


以下の座標データを元に、正しくコーディングできているか、ランダムにテストする

In [95]:
for k in all_facilities.keys():
    print("Saving as csv:", k)
    print(len(all_facilities[k]))
    if len(all_facilities[k]) == 0:
        continue
    display(all_facilities[k][0]["geocodes"].astype(str).value_counts())
#     display(all_facilities[k][0])

Saving as csv: 保育園
1


{'northeast': {'lat': 35.7035542, 'lng': 140.0112155}, 'southwest': {'lat': 35.6894305, 'lng': 139.9974705}}                              2
{'northeast': {'lat': 35.71918618029149, 'lng': 140.0208115802915}, 'southwest': {'lat': 35.71648821970849, 'lng': 140.0181136197085}}    1
{'northeast': {'lat': 35.7152334802915, 'lng': 139.9716631802915}, 'southwest': {'lat': 35.7125355197085, 'lng': 139.9689652197085}}      1
{'northeast': {'lat': 35.7367117802915, 'lng': 140.0414424802915}, 'southwest': {'lat': 35.7340138197085, 'lng': 140.0387445197085}}      1
{'northeast': {'lat': 35.7550585802915, 'lng': 140.0377016802915}, 'southwest': {'lat': 35.7523606197085, 'lng': 140.0350037197085}}      1
                                                                                                                                         ..
{'northeast': {'lat': 35.74014028029149, 'lng': 139.9807984802915}, 'southwest': {'lat': 35.73744231970849, 'lng': 139.9781005197085}}    1
{'northeast': {'lat'

Saving as csv: 幼稚園
1


{'northeast': {'lat': 35.7227269802915, 'lng': 139.9978477802915}, 'southwest': {'lat': 35.72002901970851, 'lng': 139.9951498197085}}     1
{'northeast': {'lat': 35.75582728029149, 'lng': 140.0387533802915}, 'southwest': {'lat': 35.75312931970849, 'lng': 140.0360554197085}}    1
{'northeast': {'lat': 35.7296585802915, 'lng': 140.0325641802915}, 'southwest': {'lat': 35.7269606197085, 'lng': 140.0298662197085}}      1
{'northeast': {'lat': 35.7122926802915, 'lng': 139.9946850802915}, 'southwest': {'lat': 35.70959471970851, 'lng': 139.9919871197085}}     1
{'northeast': {'lat': 35.74316708029149, 'lng': 140.0412835802915}, 'southwest': {'lat': 35.74046911970849, 'lng': 140.0385856197085}}    1
{'northeast': {'lat': 35.70478488029149, 'lng': 140.0556099802915}, 'southwest': {'lat': 35.70208691970849, 'lng': 140.0529120197085}}    1
{'northeast': {'lat': 35.73333128029149, 'lng': 140.0137282802915}, 'southwest': {'lat': 35.7306333197085, 'lng': 140.0110303197085}}     1
{'northeast': {'lat'

Saving as csv: 認定こども園
1


...

Saving as csv: 児童ホーム
1


{'northeast': {'lat': 35.73725478029149, 'lng': 140.0138270802915}, 'southwest': {'lat': 35.73455681970849, 'lng': 140.0111291197085}}    1
{'northeast': {'lat': 35.7339602802915, 'lng': 140.0596460802915}, 'southwest': {'lat': 35.7312623197085, 'lng': 140.0569481197085}}      1
{'northeast': {'lat': 35.72529288029151, 'lng': 140.0230649802915}, 'southwest': {'lat': 35.72259491970851, 'lng': 140.0203670197085}}    1
{'northeast': {'lat': 35.7232664302915, 'lng': 140.0500847302915}, 'southwest': {'lat': 35.7205684697085, 'lng': 140.0473867697085}}      1
{'northeast': {'lat': 35.70246098029149, 'lng': 139.9763506802915}, 'southwest': {'lat': 35.6997630197085, 'lng': 139.9736527197085}}     1
{'northeast': {'lat': 35.6990529802915, 'lng': 139.9963831802915}, 'southwest': {'lat': 35.69635501970851, 'lng': 139.9936852197085}}     1
{'northeast': {'lat': 35.6977265802915, 'lng': 140.0507410802915}, 'southwest': {'lat': 35.6950286197085, 'lng': 140.0480431197085}}      1
{'northeast': {'lat'

Saving as csv: 子育て支援センター
1


{'northeast': {'lat': 35.69850168029149, 'lng': 139.9785855802915}, 'southwest': {'lat': 35.69580371970849, 'lng': 139.9758876197085}}    1
{'northeast': {'lat': 35.7362657802915, 'lng': 140.0300387802915}, 'southwest': {'lat': 35.7335678197085, 'lng': 140.0273408197085}}      1
Name: geocodes, dtype: int64

Saving as csv: その他預け先
0


- エラーなくデータが取得できている？

- 保育園の住所に重複が見られる
    - `{'northeast': {'lat': 35.7035542, 'lng': 140.0112155}, 'southwest': {'lat': 35.6894305, 'lng': 139.9974705}}`

In [96]:
for k in all_facilities.keys():
    print("Saving as csv:", k)
    print(len(all_facilities[k]))
    if len(all_facilities[k]) == 0:
        continue
    all_facilities[k][0].to_csv(f"{k}.csv", index=False)
#     display(all_facilities[k][0])

Saving as csv: 保育園
1
Saving as csv: 幼稚園
1
Saving as csv: 認定こども園
1
Saving as csv: 小規模保育事業所
1
Saving as csv: 家庭的保育者
1
Saving as csv: 事業所内保育事業所
0
Saving as csv: 認証保育所
1
Saving as csv: 認可外保育施設
1
Saving as csv: 一時保育
1
Saving as csv: 病児・病後児保育
1
Saving as csv: 児童ホーム
1
Saving as csv: 子育て支援センター
1
Saving as csv: その他預け先
0


In [106]:
all_facilities["保育園"][0][all_facilities["保育園"][0]["geocodes"].astype(str).duplicated(keep=False)]

,施設名,住所,電話番号(代表),geocodes
82,東船橋ちとせ保育園,〒273-0002 船橋市東船橋1-16-25,電話：047-423-5717 FAX：047-423-5718,"{'northeast': {'lat': 35.7035542, 'lng': 140.0..."
93,グリュック保育園,〒273-0002 船橋市東船橋1-26-10,電話：047-424-3741 FAX:047-455-3457,"{'northeast': {'lat': 35.7035542, 'lng': 140.0..."


この2件だけ同じgeocodingしていました。

なんでや...?